# SMS Spam Filter with Naive Bayes

Many email and SMS frameworks have built in spam filters to help users sift through unwanted messages. A rudimentary spam filter can be created with something called a Naive Bayes algorithm. In general it uses probabilites to predict whether a message is spam or not spam based on it's contents and how they compare to previously classified messages.

There are a few steps needed to create such a filtering algorithm. The computer must:
1. Learn how humans classify messages
2. Use that estimate probabilities for new messages
3. Classify new messages based on a probability threshold.
    3. Get human clarification if needed.
    
The first step requires some data for the computer to learn about. In this case, a [data set](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection) put together by Tiago A. Almeida and Jose Maria Hidalgo and containing 5,572 SMS messages, pre-classified by humans will work well. Additional information of the data can be found [here](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition).

### Goal

The goal of this project is to creat a spam filter using a Naive Bayes algorithm that is able to correctly predict and categorize messages with at least 80% accuracy.

## Exploring the Data

In [1]:
import pandas as pd

In [2]:
sms = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])

sms.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [3]:
print(sms.shape)
sms['Label'].value_counts(dropna=False)

(5572, 2)


ham     4825
spam     747
Name: Label, dtype: int64

Cracking open the dataset reveals a few things. There are two columns, now labeled as 'Label' and 'SMS' respectively denoting whether a message is spam or ham (non-spam) and what the message is. Out of the 5572 total entries, 747 of them are spam, or a little more than 13%.

## Training and Test Set

Once we have created the filter we should test it to ensure that it is accurate. To avoid creating a test that is biased towards the filter, it is best to create that test before seeing how our filter works.

It is also desirable to retain a portion of the data strictly for testing, called a test set. The majority of the data will be used for training our algorithm, called the training set. The following bits of code will be splitting our data into a training set and test set after randomizing it.

In [4]:
sms = sms.sample(frac=1, random_state=1)

In [5]:
split_index = round(len(sms) * .8)
print(split_index)

4458


In [6]:
training = sms[:split_index].reset_index(drop=True)
print(training.shape)
training['Label'].value_counts(normalize=True)

(4458, 2)


ham     0.86541
spam    0.13459
Name: Label, dtype: float64

In [7]:
test = sms[split_index:].reset_index(drop=True)
print(test.shape)
test['Label'].value_counts(normalize=True)

(1114, 2)


ham     0.868043
spam    0.131957
Name: Label, dtype: float64

The data set has now been split, and the ratio of spam and non-spam messages is within the same percentage as the original data set. This means the training and testing sets are representative of the original data.

## Cleaning Data

With the data split into a test and training set, it is worth cleaning the data to make analysis easier in future steps. Namely, all the data should be lower-case and all punctuation should be removed.

#### Capitalization & Punctuation

In [8]:
training.head(3)  #before

,Label,SMS
0,ham,"Yep, by the pretty sculpture"
1,ham,"Yes, princess. Are you going to make me moan?"
2,ham,Welp apparently he retired


In [9]:
training['SMS'] = training['SMS'].str.replace('\W', ' ').str.lower()
training.head(3)  #after

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired


#### Creating Vocabulary

This step will transform the SMS message into a datafram with each column (aside from the Label column) representing a unique word in the vocabulary. Then, for each row of the datafram, the column will contain a frequency of that word in the original message.

The first step is to split the SMS into individual words.

In [10]:
training['SMS'] = training['SMS'].str.split()

vocabulary = []
for message in training['SMS']:
    for word in message:
        vocabulary.append(word)

vocabulary = list(set(vocabulary)) #remove duplicate items

Next the vocabulary will be converted to keys of a dictionary and their values will be a list containing the number of times the word occured in a message. The 0th index of that list corresponds to the 0th message, 1st index to 1st message, and so forth.

In [11]:
word_counts_per_sms = {unique_word: [0] * len(training['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

#### Finalized Training Set

The last step is to convert the dictionary into a Dataframe, then stitch it onto the original.

In [12]:
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,0,00,000,000pes,008704050406,0089,01223585334,02,0207,02072069400,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [13]:
f_train = pd.concat([training, word_counts], axis=1)
f_train.head()

,Label,SMS,0,00,000,000pes,008704050406,0089,01223585334,02,...,zindgi,zoe,zogtorius,zouk,zyada,é,ú1,ü,〨ud,鈥
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0


In [14]:
print(f_train.shape)

(4458, 7785)


The finalized training data contains 7785 unique words accross 4458 messages; some of which are numbers and special characters.

## Creating the Spam Filter

With clean training data, the next task is to create the filter itself. In order to classify new messages, the Naive Bayes algorithm must be able to calculate the probabilities of getting spam or ham given the message contains certain words. In more mathematical terms, **equations 1 and 2:**

\begin{equation}
P(Spam \ | \ \omega_1, \omega_2, ..., \omega_n) \propto P(Spam) \cdot \prod^{n}_{i = 1} P(\omega_i \ | \ Spam)
\end{equation}

\begin{equation}
P(Ham \ | \ \omega_1, \omega_2, ..., \omega_n) \propto P(Ham) \cdot \prod^{n}_{i = 1} P(\omega_i \ | \ Ham)
\end{equation}


Furthermore, **equations 3 and 4:**

$$ P(\omega_i \ | \ Spam) = \frac{N_{\omega_i | Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}} $$


$$ P(\omega_i \ | \ Ham) = \frac{N_{\omega_i | Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}} $$

### Constants

Some of the values in the above equations will remain the same for each new message. This means we can calculate them ahead of time. The values that are constants are:
- P(Spam) -- The probability of getting Spam
- P(Ham) -- The probability of getting Non-Spam
- N<sub>Spam</sub> -- Number of words in *all* spam messages
- N<sub>Ham</sub> -- Number of words in *all* non-spam messages
- N<sub>Vocabulary</sub> -- Number of words in *all* the vocabulary
- $\alpha$ -- A smoothing factor of 1

Let's calculate them below to be used a bit later.

In [37]:
#Separate spam and ham messages
spam = f_train[ f_train['Label'] == 'spam' ]
ham = f_train[ f_train['Label'] == 'ham' ]

#calculate p(spam) and p(ham)
p_spam = len(spam) / len(f_train)
p_ham = len(ham) / len(f_train)

#sum the amount of words in each message
n_spam = spam['SMS'].apply(len).sum()
n_ham = ham['SMS'].apply(len).sum()

#vocabulary created when cleaning data
n_vocab = len(vocabulary)

#smoothing factor
alpha = 1

### Calculating Parameters

The terms left in the last two equations that are not constants will depend on the individual words found in each message. Namely, $ N_{\omega_i \ | \ Spam} $ and $ N_{\omega_i \ | \ Ham} $. Fortunately, these values can be calculated once, prior to testing new messages. This is because the probabilities are calculated from the training set, which remains unchanged through this analysis. Since there will be 2 probabilites calculated per word (one for spam, one for ham) and there are 7785 unique words in the vocabulary, there will be 15,566 probabilities to calculate.

In [38]:
spam_param = {}
ham_param = {}

for word in vocabulary:
    #spam parameters
    s_numerator = spam[word].sum() + alpha
    s_denominator = n_spam + alpha * n_vocab
    spam_param[word] = s_numerator / s_denominator
    
    #ham parameters
    h_numerator = ham[word].sum() + alpha
    h_denominator = n_ham + alpha * n_vocab
    ham_param[word] = h_numerator / h_denominator
    

### Handling New Messages

With the core calculations taken care of as overhead, the spam filter now must start predeciting if new messages are spam or ham. This can be broken down into the following steps:
1. Take in a new message as input
2. Calculate $ P(Spam \ | \ \omega_1, \omega_2, ..., \omega_n) $ and $ P(Ham \ | \ \omega_1, \omega_2, ..., \omega_n) $
3. Compare the two values and classify as spam or ham based on which probability is larger.
A. Request human clarification if the probabilities are tied.

These steps will be carried out in a single function.
- `verbose=True` will print the results to screen
- `return_labels=True` will return the results from the function call

In [61]:
import re

def classify(message, verbose=True, return_labels=False):
    message = re.sub('\W', ' ', message).lower().split()
    
    #initialize as p(spam) and p(ham) for multiplication
    #before product notation (capital pi) in equations 1 & 2
    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    #parse each word and multiply into probability
    for word in message:
        if word in spam_param:
            p_spam_given_message *= spam_param[word]
        if word in ham_param:
            p_ham_given_message *= ham_param[word]

    #set classification
    if p_spam_given_message > p_ham_given_message:
        answer = 'spam'
    elif p_spam_given_message < p_ham_given_message:
        answer = 'ham'
    else:
        answer = 'human'
    
    
    if verbose:
        print('P(Spam|message): ', p_spam_given_message)
        print('P( Ham|message): ', p_ham_given_message)
        
        if answer == 'spam':
            print('Label: Spam')
        elif answer == 'ham':
            print('Label: Ham')
        else:
            print('Requires Human Classification')

            
    if return_labels:
        if answer == 'spam':
            return 'spam'
        elif answer == 'ham':
            return 'ham'
        else:
            return 'Requires Human Classification'
        

In [62]:
obv_spam = 'WINNER!! THIS IS THE SECRET CODE TO UNLOCK the MONEY: c3421.'
obv_not = 'Hello world, this is a test'

classify(obv_spam)
classify(obv_not)

P(Spam|message):  1.3481290211300841e-25
P( Ham|message):  1.9368049028589875e-27
Label: Spam
P(Spam|message):  2.1944441392195382e-20
P( Ham|message):  3.0077368441603413e-17
Label: Ham


With a very simplified check, these answers do look promising.

## Verify Filter Accuracy

It is now time to check the accuracy of the spam filter on the test set. A new column in the test set will be created with the output of the function. This will require the `return_labels` argument set to `True` and it's best if `verbose` argument is set to `False`.

In [64]:
test['predicted'] = test['SMS'].apply(classify, verbose=False, return_labels=True)
test.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


With the new column created, accuracy can be calculated as follows:

\begin{equation}
\textrm{Accuracy} = \frac{\textrm{Number of correct classifications}}{\textrm{Total classifications}}
\end{equation}

In [74]:
n_correct = 0

for row in test.iterrows():
    row = row[1]
    if row['predicted'] == row['Label']:
        n_correct += 1

accuracy = n_correct / len(test)

print('Correct Classifications: ', n_correct)
print('Incorrect Classifications: ', len(test) - n_correct)
print('Total Classifications: ', len(test))
print('\nAccuracy: ', accuracy)

Correct Classifications:  1100
Incorrect Classifications:  14
Total Classifications:  1114

Accuracy:  0.9874326750448833


#### Results

The spam filter correctly classified 98.74% of spam messages. This seems like a great start, especially with a simplified data set and probabilistic approach! This would likely be less accurate with a diversified vocabulary but overall very strong results for the task at hand.

## Potential Next Steps

- Isolate the incorrect messages and examine the wrong conclusion
- Make filtering algorithm case sensitive